In [1]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import argparse

def test_model_accuracy(model_path, dataset_path, image_size=(224, 224), batch_size=32):
    # Load model
    model = load_model(model_path)

    # Prepare data
    datagen = ImageDataGenerator(rescale=1./255)

    test_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )

    # Predict
    predictions = model.predict(test_generator)
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
    true_classes = test_generator.classes

    # Accuracy
    accuracy = accuracy_score(true_classes, predicted_classes)
    print(f"\n✅ Model Accuracy on Test Set: {accuracy * 100:.2f}%")


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

dataset_path = "/content/drive/MyDrive/Data_set"

# Check subfolders inside train directory
print("🔍 Folders inside dataset:")
print(os.listdir(dataset_path))

🔍 Folders inside dataset:
['Cancer', 'Non_cancer', 'Cnace_Size', 'Non_Cancer_Size']


In [4]:
import shutil

unwanted_folders = [".ipynb_checkpoints", "other"]
for folder in unwanted_folders:
    folder_path = os.path.join(dataset_path, folder)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"🗑 Deleted: {folder}")

print("✅ Dataset cleaned!")

✅ Dataset cleaned!


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import tensorflow as tf

def test_model_accuracy(model_path, dataset_path, batch_size=32):
    try:
        print(f"🔍 Loading model from: {model_path}")
        model = load_model(model_path, compile=False)

        # Detect model input image size
        image_size = model.input_shape[1:3]
        print(f"📐 Detected input size: {image_size}")

        # Prepare test data generator
        datagen = ImageDataGenerator(rescale=1./255)
        test_generator = datagen.flow_from_directory(
            dataset_path,
            target_size=image_size,
            batch_size=batch_size,
            class_mode='binary',
            shuffle=False
        )

        print(f"🧪 Testing on {len(test_generator.filenames)} images...")

        # Predict
        predictions = model.predict(test_generator, verbose=1)
        predicted_classes = (predictions > 0.5).astype("int32").flatten()
        true_classes = test_generator.classes

        # Calculate accuracy
        accuracy = accuracy_score(true_classes, predicted_classes)
        print(f"\n✅ Model Accuracy: {accuracy * 100:.2f}%")

    except Exception as e:
        print("🚨 An error occurred while testing the model:")
        print(str(e))


# EXAMPLE USAGE (you can change these paths)
if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/one.h5"
    dataset_path = "/content/drive/MyDrive/Data_set"

    test_model_accuracy(model_path, dataset_path)


🔍 Loading model from: /content/drive/MyDrive/one.h5
📐 Detected input size: (224, 224)
Found 7865 images belonging to 4 classes.
🧪 Testing on 7865 images...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


246/246 ━━━━━━━━━━━━━━━━━━━━ 2102s 9s/step

✅ Model Accuracy: 22.12%


In [ ]:
if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/small_data.h5"
    dataset_path = "/content/drive/MyDrive/Data_set"

    test_model_accuracy(model_path, dataset_path)


In [ ]:
# import cv2
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import matplotlib
# matplotlib.use('Agg')  # prevent GUI backend issues
# import os

# def generate_visual_prediction(model_path, image_path, output_path='xai_output.png'):
#     # Load model
#     print(f"🔍 Loading model from: {model_path}")
#     model = load_model(model_path, compile=False)
#     image_size = model.input_shape[1:3]

#     # Load image
#     img = cv2.imread(image_path)
#     original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     resized_img = cv2.resize(original_img, image_size)
#     input_array = np.expand_dims(resized_img / 255.0, axis=0)

#     # Predict
#     pred = model.predict(input_array)[0][0]
#     label = "NON_CANCER" if pred > 0.5 else "CANCER"
#     confidence = float(pred) if label == "CANCER" else 1 - float(pred)
#     confidence_percent = confidence * 100

#     # 🧠 Find last Conv2D layer
#     last_conv_layer = None
#     for layer in reversed(model.layers):
#         if isinstance(layer, tf.keras.layers.Conv2D):
#             last_conv_layer = layer.name
#             break
#     if last_conv_layer is None:
#         raise ValueError("❌ No Conv2D layer found in model.")

#     # Grad-CAM
#     grad_model = tf.keras.models.Model(
#         [model.inputs],
#         [model.get_layer(last_conv_layer).output, model.output]
#     )

#     with tf.GradientTape() as tape:
#         conv_outputs, predictions = grad_model(input_array)
#         loss = predictions[:, 0]

#     grads = tape.gradient(loss, conv_outputs)
#     if grads is None:
#         raise ValueError("❌ Gradient is None — unable to compute Grad-CAM.")

#     pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
#     conv_outputs = conv_outputs[0]
#     heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
#     heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
#     heatmap = heatmap.numpy()

#     # Prepare heatmap
#     heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
#     heatmap = np.uint8(255 * heatmap)
#     heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

#     # Overlay heatmap
#     overlayed_img = cv2.addWeighted(original_img, 0.6, heatmap_color, 0.4, 0)

#     # 📝 Add prediction text with confidence
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     text = f"PREDICTION: {label} ({confidence_percent:.2f}%)"
#     cv2.putText(overlayed_img, text, (10, 30), font, 1, (255, 255, 255), 2)

#     # 🖼️ Combine original and heatmap
#     combined = np.hstack((original_img, overlayed_img))
#     combined_bgr = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)
#     cv2.imwrite(output_path, combined_bgr)

#     print(f"✅ Visual output saved: {output_path}")


In [ ]:
# generate_visual_prediction(
#     model_path="/content/drive/MyDrive/one.h5",
#     image_path="/content/sample_data/images.jpeg",
#     output_path="xai_result.png"
# )


🔍 Loading model from: /content/drive/MyDrive/one.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step
✅ Enhanced Grad-CAM saved to xai_result.png


In [ ]:
# generate_visual_prediction(
#     model_path="/content/drive/MyDrive/small_data.h5",
#     image_path="/content/sample_data/images.jpeg",
#     output_path="xai_1result.png"
# )


🔍 Loading model from: /content/drive/MyDrive/small_data.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ Enhanced Grad-CAM saved to xai_1result.png


In [ ]:
import os
import random

def balance_dataset(parent_folder_path):
    cancer_dir = os.path.join(parent_folder_path, 'Cancer')
    non_cancer_dir = os.path.join(parent_folder_path, 'Non_cancer')

    # Check if both folders exist
    if not os.path.exists(cancer_dir) or not os.path.exists(non_cancer_dir):
        print("❌ 'cancer' or 'non_cancer' folder not found in given path.")
        return

    # List images
    cancer_images = [f for f in os.listdir(cancer_dir) if os.path.isfile(os.path.join(cancer_dir, f))]
    non_cancer_images = [f for f in os.listdir(non_cancer_dir) if os.path.isfile(os.path.join(non_cancer_dir, f))]

    len_cancer = len(cancer_images)
    len_non_cancer = len(non_cancer_images)

    print(f"📊 Cancer images: {len_cancer}")
    print(f"📊 Non-Cancer images: {len_non_cancer}")

    if len_cancer == len_non_cancer:
        print("✅ Dataset is already balanced.")
        return

    # Remove extra images
    if len_cancer > len_non_cancer:
        to_delete = random.sample(cancer_images, len_cancer - len_non_cancer)
        for img in to_delete:
            os.remove(os.path.join(cancer_dir, img))
    else:
        to_delete = random.sample(non_cancer_images, len_non_cancer - len_cancer)
        for img in to_delete:
            os.remove(os.path.join(non_cancer_dir, img))

    print("✅ Dataset has been balanced.")

# ⬇️ Bas yahan apna folder ka path daalo
balance_dataset("/content/drive/MyDrive/Data_set")


📊 Cancer images: 1727
📊 Non-Cancer images: 1739
✅ Dataset has been balanced.


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

def generate_visual_prediction(model_path, image_path, output_path='xai_result.png'):
    model = load_model(model_path, compile=False)
    img_size = model.input_shape[1:3]

    # Load and preprocess image
    img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb_img, img_size)
    input_tensor = np.expand_dims(resized / 255.0, axis=0)

    # Predict
    pred = model.predict(input_tensor)[0][0]
    class_idx = int(pred <= 0.5)
    confidence = pred if class_idx == 0 else 1 - pred
    label = "NON-CANCER" if class_idx == 0 else "CANCER"

    # Grad-CAM setup
    conv_layer = None
    for l in reversed(model.layers):
        if isinstance(l, tf.keras.layers.Conv2D):
            conv_layer = l.name
            break
    if not conv_layer:
        raise ValueError("Conv2D layer not found.")

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(conv_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(input_tensor)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_output)[0]
    conv_output = conv_output[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = np.zeros(conv_output.shape[:2], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_output[:, :, i]

    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (rgb_img.shape[1], rgb_img.shape[0]))
    cam = cam / (np.max(cam) + 1e-8)

    # Heatmap
    heatmap = np.uint8(255 * cam)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(rgb_img, 0.6, heatmap_color, 0.4, 0)

    # Add label and confidence - placed in bottom-right corner
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    thickness = 2
    label_text = f"{label} ({confidence*100:.2f}%)"
    text_size = cv2.getTextSize(label_text, font, font_scale, thickness)[0]
    x = overlay.shape[1] - text_size[0] - 20
    y = overlay.shape[0] - 20
    cv2.putText(overlay, label_text, (x, y), font, font_scale, (255, 255, 255), thickness)

    # Combine original and heatmap overlay
    combined = np.hstack((rgb_img, overlay))
    result = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, result)
    print(f"[✅] Output image saved to: {output_path}")

# # Example usage
# generate_visual_prediction(
#     model_path="/content/drive/MyDrive/one.h5",
#     image_path="/content/sample_data/images (2).jpeg",
#     output_path="xai_result.png"
# )


In [ ]:
generate_visual_prediction(
    model_path="/content/drive/MyDrive/one.h5",
    image_path="/content/sample_data/aug_1_C008.jpeg",
    output_path="xai_result.png"
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer_3']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


[✅] Output image saved to: xai_result.png


In [ ]:
generate_visual_prediction(
    model_path="/content/drive/MyDrive/small_data.h5",
    image_path="/content/sample_data/aug_1_C008.jpeg",
    output_path="xai_1result.png"
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 761ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


[✅] Output image saved to: xai_1result.png
